In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# ======================
# EXERCISE CONFIGURATION
# ======================
EXERCISES = {
    "bicep_curl": {
        "landmarks": {
            "joint1": mp_pose.PoseLandmark.LEFT_SHOULDER,
            "joint2": mp_pose.PoseLandmark.LEFT_ELBOW,
            "joint3": mp_pose.PoseLandmark.LEFT_WRIST
        },
        "good_angles": (30, 160),
        "feedback_rules": {
            "elbow_stability": {"threshold": 0.1, "message": "Keep elbows stable!"},
            "wrist_alignment": {"threshold": 0.2, "message": "Don't bend wrists!"}
        }
    },
    "squat": {
        "landmarks": {
            "hip": mp_pose.PoseLandmark.LEFT_HIP,
            "knee": mp_pose.PoseLandmark.LEFT_KNEE,
            "ankle": mp_pose.PoseLandmark.LEFT_ANKLE
        },
        "good_angles": (80, 120),
        "feedback_rules": {
            "knee_over_toes": {"threshold": 0.15, "message": "Knees over toes!"},
            "back_angle": {"threshold": 0.2, "message": "Maintain neutral spine!"}
        }
    }
}

CURRENT_EXERCISE = "bicep_curl"  # Change to "squat" if needed

# ======================
# CORE LOGIC
# ======================
class ExerciseAnalyzer:
    def __init__(self):
        self.feedback = []
        self.rep_count = 0
        self.stage = None

    def calculate_angle(self, a, b, c):
        a, b, c = np.array(a), np.array(b), np.array(c)
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(np.degrees(radians))
        return angle if angle <= 180 else 360 - angle

    def check_form(self, landmarks, exercise_config):
        self.feedback = []
        config = EXERCISES[exercise_config]
        
        joints = {name: [landmarks[joint.value].x, landmarks[joint.value].y] 
                  for name, joint in config["landmarks"].items()}
        
        angle = self.calculate_angle(joints["joint1"], joints["joint2"], joints["joint3"])
        
        # Feedback based on angle range
        if not (config["good_angles"][0] <= angle <= config["good_angles"][1]):
            self.feedback.append(f"Adjust angle! Current: {angle:.1f}°")
        
        # Exercise-specific feedback
        if exercise_config == "bicep_curl":
            if abs(joints["joint1"][0] - joints["joint3"][0]) > 0.1:
                self.feedback.append(config["feedback_rules"]["elbow_stability"]["message"])

            # Curl rep counter
            if angle > 160:
                self.stage = "down"
            if angle < 30 and self.stage == 'down':
                self.stage = "up"
                self.rep_count += 1

        elif exercise_config == "squat":
            if abs(joints["knee"][0] - joints["ankle"][0]) > 0.15:
                self.feedback.append(config["feedback_rules"]["knee_over_toes"]["message"])

        return angle, self.feedback, self.rep_count, self.stage


# ======================
# MAIN EXECUTION
# ======================
analyzer = ExerciseAnalyzer()
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            continue

        # Preprocess frame
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            try:
                angle, feedback, reps, stage = analyzer.check_form(
                    results.pose_landmarks.landmark, 
                    CURRENT_EXERCISE
                )

                # Draw feedback text
                y_offset = 30
                for msg in feedback:
                    cv2.putText(image, msg, (10, y_offset), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                    y_offset += 30

                # Display angle
                elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value]
                coords = tuple(np.multiply([elbow.x, elbow.y], [640, 480]).astype(int))
                cv2.putText(image, f"{angle:.1f}°", coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

                # Render REP + STAGE counter box
                cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
                cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
                cv2.putText(image, str(reps), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)

                cv2.putText(image, 'STAGE', (65, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
                cv2.putText(image, str(stage), (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)

            except Exception as e:
                print(f"Tracking Error: {e}")

            # Draw pose landmarks
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )

        # Show output
        cv2.imshow('AI Gym Trainer', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 